In [1]:
from math import sin, cos, log
import numpy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [2]:
def f(u):
    """Возвращает численное значение в правой части 
    системы уравнений фугоидного движения
    """
    
    v = u[0]
    theta = u[1]
    x = u[2]
    y = u[3]
    return numpy.array([-g*sin(theta) - C_D/C_L*g/v_t**2*v**2,
                      -g*cos(theta)/v + g/v_t**2*v,
                      v*cos(theta),
                      v*sin(theta)])

In [3]:
#определение функции Эйлера с пересчётом
def euler_midpoint_step(u, f, dt):
    """
    Возвращает численное решение методом Эйлера на шаге n+1
    
    u: массив перменных float
    f: функция, для которой вычисляестся правая часть
    dt: шаг по времени
    
    """
    u_star = u + 0.5*dt * f(u)
    return u + dt * f(u_star)

In [4]:
def get_diffgrid(u_current, u_fine, dt):
    """
    Возвращает разность между разными решениями
    
    u_current: массив переменных типа float, текущее решение
    u_finest: массив переменных типа float, сравниваемое значение
    dt: шаг по времени
    
    diffgrid: разница, вычисленная по норме L1
    """
    
    N_current = len(u_current[:,0])
    N_fine = len(u_fine[:,0])
   
    grid_size_ratio = ceil(N_fine/N_current)
    
    diffgrid = dt * numpy.sum( numpy.abs(\
            u_current[:,2]- u_fine[::grid_size_ratio,2])) 
    
    return diffgrid

In [8]:
#модельные параметры
T = 100
g = 9.8      #g
v_t = 4.9   #триммерная скорость
C_D = 1/5  #коэффициент сопротивления или D/L
C_L = 1   

#начальные условия
v0 = 20*v_t     #начальная скорость
theta0 = numpy.pi/8 #начальный наклон траектории
x0 = 0     #горизонтальное положение
y0 = 2  #начальная высота

In [ ]:
r = 2
h = 0.001

dt_values2 = numpy.array([h, r*h, r**2*h])
u_values2 = numpy.empty_like(dt_values2, dtype=numpy.ndarray)

diffgrid2 = numpy.empty(2)

for i, dt in enumerate(dt_values2):
    
    N = int(T/dt) + 1 
    t = numpy.linspace(0.0, T, N)
    u = numpy.empty((N, 4))
    u[0] = numpy.array([v0, theta0, x0, y0])

    for n in range(N-1):
        u[n+1] = euler_midpoint_step(u[n], f, dt)
    
    u_values2[i] = u
    

#расчет разницы f2-f1
diffgrid2[0] = get_diffgrid(u_values2[1], u_values2[0], dt_values2[1])

#расчет разницы f3-f2
diffgrid2[1] = get_diffgrid(u_values2[2], u_values2[1], dt_values2[2])

#расчет порядка сходимости
p = (log(diffgrid2[1]) - log(diffgrid2[0])) / log(r)

print('The order of convergence is p = {:.3f}'.format(p));